# Chapter 3: Complete Data Integration

## Learning Objectives

By the end of this chapter, you will be able to:
- Create AI-powered location matching between different data formats
- Integrate all cleaning functions to process scraped data end-to-end
- Merge Indeed and OEWS datasets using common keys
- Produce a final clean dataset ready for dashboard creation

---

## Bringing It All Together

> **Instructor Cue:** Start with energy and urgency: "We're in the home stretch! We have salary parsing and job title matching. Now we need one more piece - location matching - and then we can merge our datasets for the final time before lunch."

We've built powerful AI tools for salary parsing and job title matching. Now let's complete the puzzle with location matching and create our final integrated dataset.

### The Location Challenge

In [ ]:
# Quick setup and data loading
import pandas as pd
from pathlib import Path
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
import nest_asyncio
import os
from dotenv import load_dotenv

# Apply nest_asyncio for Jupyter compatibility and load environment
nest_asyncio.apply()
load_dotenv()

# Set up Google AI provider
API_KEY = os.getenv("GOOGLE_API_KEY")
provider = GoogleProvider(api_key=API_KEY)
model = GoogleModel("gemini-1.5-flash", provider=provider)

# Load OEWS data
bls_file = Path("../01_module/data/bls_jobs_metro_area_2024.csv")
oews_df = pd.read_csv(bls_file)

# Load scraped Indeed data using consistent pattern
scraped_file_paths = list(Path("../01_module/data").glob("scraped_indeed_*.csv"))
dataframes = [pd.read_csv(file) for file in scraped_file_paths]
indeed_df = pd.concat(dataframes, ignore_index=True)

print(f"📊 Loaded {len(scraped_file_paths)} scraped files with {len(indeed_df)} total jobs")

print("\n=== LOCATION MATCHING CHALLENGE ===")
print("Indeed locations (sample):")
for location in indeed_df["location"].unique()[:5]:
    print(f"  • {location}")

print("\nOEWS AREA_TITLE (sample):")
for area in oews_df["AREA_TITLE"].unique()[:5]:
    print(f"  • {area}")

print("\nOEWS PRIM_STATE (sample):")
print(f"  States: {sorted(oews_df['PRIM_STATE'].unique())}")

> **Instructor Cue:** Point out that Indeed has "Portland, OR" while OEWS has "Portland-Vancouver-Hillsboro, OR-WA" for AREA_TITLE and "OR" for PRIM_STATE. Ask: "How would we match these automatically?"

---

## Creating Location Matching Function

Let's build our final AI function to match locations between datasets.

In [ ]:
# Model for a SINGLE location match. Added 'original_location' for easier mapping.
class LocationMatch(BaseModel):
    original_location: str = Field(
        ..., description="The original location string from the input list."
    )
    matched_area: str = Field(
        ..., description="Best matching AREA_TITLE from the provided OEWS options."
    )
    matched_state: str = Field(
        ..., description="Best matching PRIM_STATE from the provided OEWS options."
    )
    confidence: int = Field(
        ..., description="A 1-10 confidence score for the overall match quality."
    )
    reasoning: str = Field(..., description="A brief explanation for the match decision.")


# Model for a BATCH of location matches
class LocationMatchResults(BaseModel):
    matches: list[LocationMatch]


# The Agent is created once and configured for the batch location matching task.
# This avoids the inefficiency of creating a new agent on every function call.
_location_matcher = Agent(
    "gemini-1.5-flash",  # Assumes 'model' is a pre-configured GoogleModel instance
    output_type=LocationMatchResults,
    system_prompt="""
    You are an expert at matching a list of informal geographic locations to official US government area and state names.

    Your task: For each input location, find the best matching official OEWS metropolitan statistical area (AREA_TITLE) and primary state (PRIM_STATE) from the provided lists.

    Guidelines:
    1. Match each location from the input list to the best available AREA_TITLE and PRIM_STATE.
    2. AREA_TITLE is a metropolitan area (e.g., "Portland-Vancouver-Hillsboro, OR-WA").
    3. PRIM_STATE is a 2-letter state code (e.g., "OR", "TX", "CA").
    4. If no reasonable area match is found, use "Unknown" for matched_area.
    5. Always attempt to match the state, even if the area is unknown.
    6. For general terms like "Remote" or "United States", both area and state should be "Unknown".
    7. Provide a confidence score (1-10) and a brief reasoning for each match.
    """,
)


def ai_match_location2msa(
    indeed_locations: list[str], oews_areas: list[str], oews_states: list[str]
) -> list[dict]:
    """
    Uses a single AI batch call to match a list of Indeed locations to OEWS areas and states.

    Args:
        indeed_locations: A list of location strings from Indeed.
        oews_areas: A complete list of AREA_TITLE values from OEWS.
        oews_states: A complete list of PRIM_STATE values from OEWS.

    Returns:
        A list of dictionaries, each containing the match result for a location.
    """
    # Filter out any empty or null locations before processing
    valid_locations = [loc for loc in indeed_locations if pd.notna(loc) and str(loc).strip()]
    if not valid_locations:
        return []

    # Create the context and the list of items to match for the prompt
    area_options = "\n".join([f"- {area}" for area in oews_areas])
    state_options = "\n".join([f"- {state}" for state in sorted(oews_states)])
    locations_to_match = "\n".join([f'- "{loc}"' for loc in valid_locations])

    prompt = f"""
    Please process the entire list of "Locations to Match" below. For each one, find the best fit from the provided official options.

    Official AREA_TITLE Options:
    {area_options}
    - Unknown

    Official PRIM_STATE Options:
    {state_options}
    - Unknown

    Locations to Match:
    {locations_to_match}

    Return a structured list containing a match object for every location in the "Locations to Match" list.
    """

    try:
        result = _location_matcher.run_sync(prompt)
        return [match.model_dump() for match in result.output.matches]
    except Exception as e:
        print(f"A critical AI error occurred during location matching: {e}")

        return [
            {
                "original_location": loc,
                "matched_area": "Error",
                "matched_state": "Error",
                "confidence": 0,
                "reasoning": f"AI processing failed: {str(e)}",
            }
            for loc in valid_locations
        ]

---

## Testing Location Matching

In [ ]:
unique_areas = oews_df["AREA_TITLE"].unique().tolist()
unique_states = oews_df["PRIM_STATE"].unique().tolist()

# A list of test locations to match.
test_locations = [
    "Portland, OR",
    "New Orleans, LA",
    "Austin, TX",
    "Bend, OR",
    "Remote Work",
    "San Francisco Bay Area",
    "USA",
]

print("=== TESTING BATCH LOCATION MATCHING ===")
print(f"{'Indeed Location':<30} | {'Matched Area':<45} | {'State':<6} | {'Conf':<4} | Reasoning")
print("-" * 125)

# Call the batch function once with the entire list of locations.
all_matched_results = ai_match_location2msa(test_locations, unique_areas, unique_states)

# --- 3. Loop Through the Results and Print ---
# Iterate through the list of result dictionaries to display each match.
for result in all_matched_results:
    # Use .get() for safe access to dictionary keys, providing defaults.
    original = result.get("original_location", "N/A")
    area = result.get("matched_area", "N/A")
    state = result.get("matched_state", "N/A")
    confidence = result.get("confidence", 0)
    reasoning = result.get("reasoning", "N/A")

    # Use string slicing on the area name to keep the table neatly aligned.
    print(f"{original:<30} | {area[:44]:<45} | {state:<6} | {confidence:<4} | {reasoning}")

> **Instructor Cue:** Watch the AI work! Point out successful matches and discuss any interesting decisions. This completes our AI toolkit.

---

## Adding Location Function to Janitor

In [ ]:
%%writefile -a ../workshoplib/src/workshoplib/janitor.py

# Model for a SINGLE location match. Added 'original_location' for easier mapping.
class LocationMatch(BaseModel):
    original_location: str = Field(
        ..., description="The original location string from the input list."
    )
    matched_area: str = Field(
        ..., description="Best matching AREA_TITLE from the provided OEWS options."
    )
    matched_state: str = Field(
        ..., description="Best matching PRIM_STATE from the provided OEWS options."
    )
    confidence: int = Field(
        ..., description="A 1-10 confidence score for the overall match quality."
    )
    reasoning: str = Field(..., description="A brief explanation for the match decision.")


# Model for a BATCH of location matches
class LocationMatchResults(BaseModel):
    matches: list[LocationMatch]


# --- 2. Create a Reusable, Global AI Agent ---

# The Agent is created once and configured for the batch location matching task.
# This avoids the inefficiency of creating a new agent on every function call.
_location_matcher = Agent(
    "gemini-1.5-flash",  # Assumes 'model' is a pre-configured GoogleModel instance
    output_type=LocationMatchResults,
    system_prompt="""
    You are an expert at matching a list of informal geographic locations to official US government area and state names.

    Your task: For each input location, find the best matching official OEWS metropolitan statistical area (AREA_TITLE) and primary state (PRIM_STATE) from the provided lists.

    Guidelines:
    1. Match each location from the input list to the best available AREA_TITLE and PRIM_STATE.
    2. AREA_TITLE is a metropolitan area (e.g., "Portland-Vancouver-Hillsboro, OR-WA").
    3. PRIM_STATE is a 2-letter state code (e.g., "OR", "TX", "CA").
    4. If no reasonable area match is found, use "Unknown" for matched_area.
    5. Always attempt to match the state, even if the area is unknown.
    6. For general terms like "Remote" or "United States", both area and state should be "Unknown".
    7. Provide a confidence score (1-10) and a brief reasoning for each match.
    """,
)


def ai_match_location2msa(
    indeed_locations: list[str], oews_areas: list[str], oews_states: list[str]
) -> list[dict]:
    """
    Uses a single AI batch call to match a list of Indeed locations to OEWS areas and states.

    Args:
        indeed_locations: A list of location strings from Indeed.
        oews_areas: A complete list of AREA_TITLE values from OEWS.
        oews_states: A complete list of PRIM_STATE values from OEWS.

    Returns:
        A list of dictionaries, each containing the match result for a location.
    """
    # Filter out any empty or null locations before processing
    valid_locations = [loc for loc in indeed_locations if pd.notna(loc) and str(loc).strip()]
    if not valid_locations:
        return []

    # Create the context and the list of items to match for the prompt
    area_options = "\n".join([f"- {area}" for area in oews_areas])
    state_options = "\n".join([f"- {state}" for state in sorted(oews_states)])
    locations_to_match = "\n".join([f'- "{loc}"' for loc in valid_locations])

    prompt = f"""
    Please process the entire list of "Locations to Match" below. For each one, find the best fit from the provided official options.

    Official AREA_TITLE Options:
    {area_options}
    - Unknown

    Official PRIM_STATE Options:
    {state_options}
    - Unknown

    Locations to Match:
    {locations_to_match}

    Return a structured list containing a match object for every location in the "Locations to Match" list.
    """

    try:
        result = _location_matcher.run_sync(prompt)
        return [match.model_dump() for match in result.output.matches]
    except Exception as e:
        print(f"A critical AI error occurred during location matching: {e}")

        return [
            {
                "original_location": loc,
                "matched_area": "Error",
                "matched_state": "Error",
                "confidence": 0,
                "reasoning": f"AI processing failed: {str(e)}",
            }
            for loc in valid_locations
        ]

In [ ]:
import nest_asyncio
from dotenv import load_dotenv
import pandas as pd

# Load environment variables (like your API key) from a .env file
load_dotenv()

# Apply a patch to allow nested asyncio event loops, which is helpful in notebooks
nest_asyncio.apply()

# Update function names to match the latest batch-processing versions
from workshoplib.janitor import ai_parse_salaries, ai_match_job_titles, ai_match_location2msa

# Update the printed list of available functions to be accurate
print("✅ Complete janitor module ready!")
print("Available functions:")
print("  • traditional_parse_salaries (rule-based batch processing)")
print("  • ai_parse_salaries (AI-powered batch processing)")
print("  • ai_match_job_titles (AI-powered batch job title matching)")
print("  • ai_match_locations_to_msa_batch (AI-powered batch location matching)")

# --- Update the test section to use the batch function ---
# Assuming 'oews_df' is a pre-loaded pandas DataFrame
unique_areas = oews_df["AREA_TITLE"].unique()
unique_states = oews_df["PRIM_STATE"].unique()
test_locations = ["Portland, OR", "San Jose, CA", "Remote"]

print(f"\nTesting batch location matching with: {test_locations}")

# Call the batch function ONCE, which is more efficient
results = ai_match_location2msa(test_locations, unique_areas, unique_states)

# Loop through the returned list of result dictionaries to display them
for result in results:
    original_loc = result.get("original_location", "N/A")
    matched_area = result.get("matched_area", "N/A")
    matched_state = result.get("matched_state", "N/A")
    confidence = result.get("confidence", 0)

    print(f"  '{original_loc}' → '{matched_area}', {matched_state} (confidence: {confidence}/10)")

---

## Complete Data Cleaning Pipeline

Now let's use all our functions together to clean our scraped Indeed data.

> **Instructor Cue:** This is the payoff moment! Say: "Watch how all our work comes together. We're about to clean messy scraped data in just a few lines of code using our AI toolkit."

In [ ]:
# Clean our Indeed data using all our janitor functions
def clean_indeed_data(indeed_df, oews_df):
    """
    Apply all our cleaning functions to Indeed data using efficient batch processing.

    Args:
        indeed_df: Raw scraped Indeed data
        oews_df: OEWS reference data

    Returns:
        Cleaned Indeed DataFrame ready for merging
    """
    print("🧹 Starting complete data cleaning pipeline...")

    # Start with a copy
    clean_df = indeed_df.copy()

    # 1. Clean salary data using AI batch processing
    print("  Step 1: AI batch salary parsing...")
    salary_list = clean_df["salary"].dropna().tolist()
    if salary_list:
        salary_results = ai_parse_salaries(salary_list)

        # Create mapping from salary string to result
        salary_mapping = {}
        for i, salary in enumerate(salary_list):
            if i < len(salary_results):
                salary_mapping[salary] = salary_results[i]

        # Apply to dataframe
        for idx, row in clean_df.iterrows():
            if pd.notna(row["salary"]) and row["salary"] in salary_mapping:
                result = salary_mapping[row["salary"]]
                clean_df.at[idx, "min_salary"] = result.get("min_salary")
                clean_df.at[idx, "max_salary"] = result.get("max_salary")
                clean_df.at[idx, "salary_type"] = result.get("salary_type")
                clean_df.at[idx, "salary_confidence"] = result.get("confidence")

    # 2. Match job titles using AI batch processing
    print("  Step 2: AI batch job title matching...")
    target_titles = oews_df["OCC_TITLE"].unique().tolist()

    # Get unique titles and process in batch
    unique_titles = clean_df["job_title"].dropna().unique().tolist()
    if unique_titles:
        title_results = ai_match_job_titles(unique_titles, target_titles)

        # Create mapping from original title to result
        title_mapping = {}
        for result in title_results:
            original = result.get("original_title")
            if original:
                title_mapping[original] = result

        # Apply to dataframe
        for idx, row in clean_df.iterrows():
            if pd.notna(row["job_title"]) and row["job_title"] in title_mapping:
                match_data = title_mapping[row["job_title"]]
                clean_df.at[idx, "bls_title"] = match_data.get("matched_title")
                clean_df.at[idx, "title_confidence"] = match_data.get("confidence")
                clean_df.at[idx, "title_reasoning"] = match_data.get("reasoning")

    # 3. Match locations using AI (individual calls for now - could be batched later)
    print("  Step 3: AI location matching...")
    unique_areas = oews_df["AREA_TITLE"].unique()
    unique_states = oews_df["PRIM_STATE"].unique()

    unique_locations = clean_df["location"].dropna().unique().tolist()

    # Process all locations in a single batch call
    location_results = ai_match_location2msa(unique_locations, unique_areas, unique_states)

    # Create mapping from original location to result
    location_mapping = {}
    for result in location_results:
        original = result.get("original_location")
        if original:
            location_mapping[original] = result

    # Apply to dataframe
    for idx, row in clean_df.iterrows():
        if pd.notna(row["location"]) and row["location"] in location_mapping:
            match_data = location_mapping[row["location"]]
            clean_df.at[idx, "matched_area"] = match_data.get("matched_area")
            clean_df.at[idx, "matched_state"] = match_data.get("matched_state")
            clean_df.at[idx, "location_confidence"] = match_data.get("confidence")

    print("✅ Data cleaning complete!")
    return clean_df


# Clean our data
indeed_clean = clean_indeed_data(indeed_df, oews_df)

print("\n=== CLEANING RESULTS ===")
print(f"Jobs processed: {len(indeed_clean)}")
print(f"Jobs with salary data: {indeed_clean['min_salary'].notna().sum()}")
print(f"Jobs with BLS title matches: {indeed_clean['bls_title'].notna().sum()}")
print(f"Jobs with location matches: {indeed_clean['matched_area'].notna().sum()}")

---

## Final Dataset Merge

Now for the moment we've been building toward - merging Indeed and OEWS data!

In [ ]:
# Merge Indeed and OEWS data
def merge_datasets(indeed_clean, oews_df):
    """
    Merge cleaned Indeed data with OEWS official statistics.

    Args:
        indeed_clean: Cleaned Indeed DataFrame
        oews_df: OEWS reference data

    Returns:
        Merged DataFrame with both datasets
    """
    print("🔗 Merging Indeed and OEWS datasets...")

    # Prepare OEWS data for merging
    oews_merge = oews_df.copy()
    oews_merge = oews_merge.rename(columns={"OCC_TITLE": "bls_title"})

    # Select key OEWS columns for the final dataset
    keep_columns = [
        "bls_title",
        "AREA_TITLE",
        "PRIM_STATE",
        "A_MEAN",
        "H_MEAN",
        "TOT_EMP",
        "JOBS_1000",
        "H_PCT25",
        "H_MEDIAN",
        "H_PCT75",
        "A_PCT25",
        "A_MEDIAN",
        "A_PCT75",
    ]

    oews_merge = oews_merge[keep_columns]

    # Merge on job title and location
    merged = pd.merge(
        indeed_clean,
        oews_merge,
        left_on=["bls_title", "matched_area", "matched_state"],
        right_on=["bls_title", "AREA_TITLE", "PRIM_STATE"],
        how="inner",
    )

    merged.columns = merged.columns.str.lower()

    print(f"  Indeed jobs before merge: {len(indeed_clean)}")
    print(f"  OEWS records: {len(oews_df)}")
    print(f"  Successfully merged: {len(merged)} records")

    return merged


# Execute the merge
final_dataset = merge_datasets(indeed_clean, oews_df)

print("\n=== FINAL DATASET OVERVIEW ===")
print(f"Shape: {final_dataset.shape}")
print("Final columns:")
import pprint as pp

pp.pprint(list(final_dataset.columns), compact=True)

# Show sample of final data
print("\nSample of merged data:")

display(final_dataset.head())

---

## Save Final Dataset

Let's save our completed dataset for the afternoon dashboard module.

In [ ]:
# Save the final clean dataset
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

final_path = DATA_DIR / "indeed_jobs_cleaned.csv"
final_dataset.to_csv(final_path, index=False)

print(f"💾 Final dataset saved: {final_path}")
print(f"📊 Records: {len(final_dataset)}")
print(f"📋 Columns: {len(final_dataset.columns)}")

# Create a quick summary
if len(final_dataset) > 0:
    summary = {
        "total_jobs": len(final_dataset),
        "unique_companies": final_dataset["company_name"].nunique(),
        "unique_bls_titles": final_dataset["bls_title"].nunique(),
        "unique_areas": final_dataset["matched_area"].nunique(),
        "jobs_with_salary": final_dataset["min_salary"].notna().sum(),
        "average_confidence": {
            "salary": final_dataset["salary_confidence"].mean(),
            "title": final_dataset["title_confidence"].mean(),
            "location": final_dataset["location_confidence"].mean(),
        },
    }

    print("\n=== FINAL SUMMARY ===")
    print(f"📈 Total jobs in final dataset: {summary['total_jobs']}")
    print(f"🏢 Unique companies: {summary['unique_companies']}")
    print(f"💼 BLS job categories: {summary['unique_bls_titles']}")
    print(f"📍 Geographic areas: {summary['unique_areas']}")
    print(f"💰 Jobs with salary data: {summary['jobs_with_salary']}")
    print("🤖 Average AI confidence scores:")
    print(f"   Salary parsing: {summary['average_confidence']['salary']:.1f}/10")
    print(f"   Title matching: {summary['average_confidence']['title']:.1f}/10")
    print(f"   Location matching: {summary['average_confidence']['location']:.1f}/10")

---

## Module 2 Complete: Ready for Lunch!

> **Instructor Cue:** End with celebration and excitement for the afternoon: "Look what we've accomplished! We've taken messy scraped data and used AI to create a professional-grade integrated dataset. After lunch, we'll turn this into an beautiful interactive dashboard!"

### 🎉 What We've Accomplished

**Module 2 Complete Success:**
- ✅ **Salary Parsing** - Traditional AND AI-powered approaches
- ✅ **Job Title Matching** - Intelligent mapping between datasets  
- ✅ **Location Matching** - Geographic standardization with AI
- ✅ **Complete Integration** - Successfully merged Indeed + OEWS data
- ✅ **Reusable Tools** - Built a complete janitor module for future use

**Technical Achievements:**
- Built 4 sophisticated data cleaning functions
- Processed real-world messy data automatically
- Created AI agents that outperform traditional rule-based approaches
- Integrated multiple data sources with common keys
- Produced a clean, analysis-ready dataset

**Key Insights Learned:**
- AI excels at handling data complexity and variations
- Structured outputs make AI reliable for data processing
- Confidence scores help identify data quality issues
- Modular functions enable rapid development and reuse

### 🍽️ Lunch Break - See You at 1:00 PM!

> **Instructor Cue:** Send them off with energy: "You've done the hard work of data integration. After lunch, we get to the fun part - building an interactive dashboard that showcases all these insights. Rest up, because this afternoon we're going to create something amazing!"

**This Afternoon: Module 3 Preview**
- Build an interactive Streamlit dashboard
- Create dynamic visualizations of our integrated data
- Add AI-powered features to the dashboard  
- Generate PDF reports
- Deploy our complete data application

**What to Bring Back:**
- Your laptop (obviously!)
- Energy for building something visual and interactive
- Questions about the data insights you want to explore

---

*End of Module 2 - Enjoy your lunch! 🌮*